In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [ ]:
import ee
import geemap
from utils import ee_utils, date_utils, plot_config
from utils.ee_utils import back_to_float

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, List, Tuple, Union

import warnings

from pathlib import Path 

warnings.filterwarnings("ignore")

In [ ]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

from validation_field_level import (
    export_feature_collection,
    compute_et_blue_value,
    calculate_et_blue_per_field,
)

## Validating ET Blue Maps with Walter Koch's Estimates

This notebook follows a three-step process:

1. **Create Yearly ET Blue Maps (2018-2022):**  
   - Aggregate dekadal ET Blue maps into monthly maps.
   - Filter the maps to include only the growing season months (April through October).
   - Sum the monthly ET Blue maps to produce yearly ET Blue maps.

2. **Calculate Yearly ET Blue Values for Potentially Irrigated Fields:**  
   - Compute ET blue in m3/ha/yr for each crop field for each year.

3. **Compare ET Blue Values with Walter Koch's Estimates:**
   - Make pretty plots

## Aggregate dekadal ET Blue maps into monthly maps

In [ ]:
ET_blue_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_landsat_10m_monthly_TG_2022"
)


ET_blue_collection = ET_blue_collection.map(lambda img: back_to_float(img, 100))

ET_blue_collection.size().getInfo()

In [ ]:
ee_utils.print_value_ranges(ET_blue_collection, "ET_blue")

In [ ]:
# ET_blue_collection_monthly = ee_utils.aggregate_to_monthly(
#     ET_blue_collection, bands=["ET_blue", "ET_blue_m3"]
# )

# ET_blue_collection_monthly.size().getInfo()

ET_blue_collection_monthly = ET_blue_collection
date_utils.print_collection_dates(ET_blue_collection_monthly)


## Filter the maps to include only the growing season months (April through October).

In [ ]:
ET_blue_collection_monthly_growing_season = ET_blue_collection_monthly.filter(ee.Filter.calendarRange(4, 10, "month"))

date_utils.print_collection_dates(ET_blue_collection_monthly_growing_season)

## Sum the monthly ET Blue maps to produce yearly ET Blue maps.

In [ ]:
# def create_yearly_sums(collection: ee.ImageCollection) -> ee.ImageCollection:
#     """
#     Create yearly sums from a collection of monthly images, setting the date
#     from the first image of each year. Years are extracted from system:time_start.

#     Args:
#         collection: Input collection with pre-filtered monthly images

#     Returns:
#         ee.ImageCollection: Collection with yearly sum images and proper dates
#     """

#     def sum_year_images(year):
#         # Get collection for this year
#         start_date = ee.Date.fromYMD(year, 1, 1)
#         end_date = ee.Date.fromYMD(year, 12, 31)
#         year_collection = collection.filterDate(start_date, end_date)

#         # Get date from first image of the year
#         first_img_date = ee.Date(
#             year_collection.sort("system:time_start", True)
#             .first()
#             .get("system:time_start")
#         )

#         return year_collection.sum().set("system:time_start", first_img_date.millis())

#     # Extract years from timestamps
#     start_year = ee.Date(collection.first().get("system:time_start")).get("year")
#     end_year = ee.Date(
#         collection.sort("system:time_start", False).first().get("system:time_start")
#     ).get("year")
#     years = ee.List.sequence(start_year, end_year)

#     # Create yearly sums with dates
#     return ee.ImageCollection.fromImages(years.map(sum_year_images))


# ET_blue_collection_yearly = create_yearly_sums(
#     ET_blue_collection_monthly_growing_season
# )

In [ ]:
# date_utils.print_collection_dates(ET_blue_collection_yearly)

In [ ]:
# ET_blue_collection_yearly.first().bandNames().getInfo()

## Compute ET blue in m3/ha/yr for each crop field for each year and export

In [ ]:
# def compute_et_blue_value(
#     feature: ee.Feature, et_blue_image: ee.Image, scale: int = 10
# ) -> ee.Feature:
#     """
#     Compute ET blue value for a single feature in m3/ha/yr.
#     Uses median check to ensure consistent irrigation signal across the field.

#     Args:
#         feature: Input field feature
#         et_blue_image: Image with ET blue values in m3
#         scale: Scale in meters for computation

#     Returns:
#         Feature with added ET_blue_m3_ha_yr property
#     """
#     geometry = feature.geometry()
#     area_ha = ee.Number(geometry.area()).divide(10000)

#     # Check if feature's NUTZUNG is in excluded list
#     excluded_crops = get_crops_to_exclude()
#     rainfed_crops = get_rainfed_reference_crops()
#     is_excluded = ee.List([*excluded_crops, *rainfed_crops]).contains(
#         feature.get("nutzung")
#     )

#     def calculate_et_blue():
#         # First check if median ET blue is positive (consistent irrigation signal)
#         median_et_blue = et_blue_image.reduceRegion(
#             reducer=ee.Reducer.median(), geometry=geometry, scale=scale, maxPixels=1e9
#         ).get("ET_blue_m3")

#         # Only calculate total if median indicates consistent irrigation
#         def compute_total():
#             total = et_blue_image.reduceRegion(
#                 reducer=ee.Reducer.sum(), geometry=geometry, scale=scale, maxPixels=1e9
#             ).get("ET_blue_m3")

#             return ee.Number(
#                 ee.Algorithms.If(
#                     ee.Algorithms.IsEqual(total, None),
#                     0,
#                     ee.Number(total).divide(area_ha).round(),
#                 )
#             )

#         return ee.Number(
#             ee.Algorithms.If(
#                 ee.Algorithms.IsEqual(median_et_blue, None),
#                 0,
#                 ee.Algorithms.If(ee.Number(median_et_blue).gt(0), compute_total(), 0),
#             )
#         )

#     et_blue_value = ee.Number(ee.Algorithms.If(is_excluded, 0, calculate_et_blue()))

#     return feature.set("ET_blue_m3_ha_yr", et_blue_value)


# def calculate_et_blue_per_field(
#     et_blue_image: ee.Image,
#     crop_fields: ee.FeatureCollection,
#     scale: Union[int, float] = 10,
# ) -> ee.FeatureCollection:
#     """
#     Calculate ET_blue in m3/ha/yr for each crop field.

#     Args:
#         et_blue_image: Image containing ET_blue estimates in m3/yr
#         crop_fields: Collection of crop field features
#         scale: Scale in meters for computations

#     Returns:
#         FeatureCollection with ET_blue_m3_ha_yr property added to all features
#     """
#     return crop_fields.map(lambda f: compute_et_blue_value(f, et_blue_image, scale))

In [ ]:
# def export_feature_collection(collection: ee.FeatureCollection, task_name: str, asset_id: str): 
#     """
#     Export the feature collection to an Earth Engine asset.

#     Args:
#         collection: The feature collection to export.
#         year: The year of the feature collection.
#         task_name: The name of the export task.
#         asset_id: The asset ID to export to.
#     """
#     task = ee.batch.Export.table.toAsset(
#         collection=collection,
#         description=task_name,
#         assetId=asset_id,
#     )
#     task.start()

In [ ]:
# years = range(2022, 2023)

# not_irrigated_crops = get_crops_to_exclude()
# exclude_filter = ee.Filter.inList("nutzung", list(not_irrigated_crops)).Not()

# ET_blue_collection_yearly_list = ET_blue_collection_yearly.toList(
#     ET_blue_collection_yearly.size()
# )

# for year in years:
#     index = year - 2022
#     nutzung_collection_year = ee.FeatureCollection(
#         f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
#     )

#     # # Create a new property nutzung same as current NUTZUNG
#     # nutzung_collection_year = nutzung_collection_year.map(
#     #     lambda feature: feature.set("nutzung", feature.get("NUTZUNG"))
#     # )

#     et_blue_image = ee.Image(ET_blue_collection_yearly_list.get(index))

#     fields_with_irrigation = calculate_et_blue_per_field(
#         et_blue_image, nutzung_collection_year
#     )

#     task_name = f"ET_blue_m3_ha_yr_per_field_from_monthly_landsat_10m_TG_{year}"
#     asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_m3_ha_yr_per_field_from_monthly_Landsat_10m_2018-2022/{task_name}"

#     export_feature_collection(fields_with_irrigation, task_name, asset_id)
#     print(f"Exporting {task_name} to {asset_id}")

In [ ]:
# palette = ["white", "blue", "green", "yellow", "red"]

# # Define visualization parameters
# visualization = {
#     "min": 0,
#     "max": 500,  # Adjust based on your maximum irrigation volume
#     "palette": palette,
# }

# # Convert irrigate_fields_w_estimates to an image for gradient visualization
# irrigation = fields_with_irrigation.reduceToImage(
#     properties=["ET_blue_m3_ha_yr"], reducer=ee.Reducer.first()
# )

# # Create a map centered on the area of interest
# Map = geemap.Map(center=[47.63915833792603, 8.77542613019931], zoom=12)

# # Add the gradient visualization layer to the map
# Map.addLayer(irrigation, visualization, "ET_blue_m3_ha_yr")

# Map

In [ ]:
# # Print feature with id =000000000000000048c7 in the nutzung_collection_year
# feature = crop_with_et_blue.filter(ee.Filter.eq("system:index", "000000000000000048c7")).first()
# feature.getInfo()

In [ ]:
# nutzung_collection_year = ee.FeatureCollection(
#     f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{2020}_area"
# )


# potentially_rainfed, _ = filter_crops(nutzung_collection_year, exclude_filter, _)

# et_blue_image = ee.Image(ET_blue_collection_yearly_list.get(2))

# crop_with_et_blue = calculate_et_blue_per_field(et_blue_image, potentially_rainfed)

## Monthly field level estimate

In [ ]:
output_prefix = "ET_blue_m3_ha_month_per_field_from_landsat_10m_TG_2022"
asset_path = "projects/thurgau-irrigation/assets/Thurgau/ET_blue_m3_ha_month_per_field_from_monthly_Landsat_10m_2018-2022"

# Get feature collection for the year
nutzung_collection_2022 = ee.FeatureCollection(
    "projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_2022_area"
)

# Irrigation threshold for ET blue
irrigation_threshold = 100  # m3/ha/month

# Start with the original features
processed_features = nutzung_collection_2022

# Convert image collection to list for processing
image_list = ET_blue_collection_monthly.toList(ET_blue_collection_monthly.size())
collection_size = image_list.size().getInfo()

# Process each image and add its values as a new property
for i in range(collection_size):
    image = ee.Image(image_list.get(i))

    # Get month for property naming
    month = ee.Date(image.get("system:time_start")).format("MM").getInfo()

    # Process the image and add as new property
    processed_features = calculate_et_blue_per_field(
        image,
        processed_features,
        output_property=f"ET_blue_m3_ha_month_{month}",
        threshold=irrigation_threshold,
    )

# Create single export task for all months
task_name = output_prefix
asset_id = f"{asset_path}/{task_name}"

# Export
task = ee.batch.Export.table.toAsset(
    collection=processed_features,
    description=task_name,
    assetId=asset_id,
)
task.start()
print(f"Exporting {task_name} to {asset_id}")

In [ ]:
processed_features.first().getInfo()

In [ ]:
palette = ["white", "blue", "green", "yellow", "red"]

# Define visualization parameters
visualization = {
    "min": 0,
    "max": 500,  # Adjust based on your maximum irrigation volume
    "palette": palette,
}

# Convert irrigate_fields_w_estimates to an image for gradient visualization
irrigation_08 = processed_features.reduceToImage(
    properties=["ET_blue_m3_ha_month_08"], reducer=ee.Reducer.first()
)
irrigation_10 = processed_features.reduceToImage(
    properties=["ET_blue_m3_ha_month_10"], reducer=ee.Reducer.first()
)

# Create a map centered on the area of interest
Map = geemap.Map(center=[47.63915833792603, 8.77542613019931], zoom=12)

# Add the gradient visualization layer to the map
Map.addLayer(irrigation_08, visualization, "ET_blue_m3_ha_month_08")
Map.addLayer(irrigation_10, visualization, "ET_blue_m3_ha_month_10")

Map

## Comparing to Walter Koch's estimates

In [ ]:
irrigation_estimate = {
    "Einjährige Freilandgemüse, ohne Konservengemüse": [200, 1000],
    "Kartoffeln": [200, 1000],
    "Freiland-Konservengemüse": [200, 600],
}

IRRIGATION_EFFICIENCY = 0.5

In [ ]:
# validator = IrrigationValidator(
#     irrigation_efficiency=IRRIGATION_EFFICIENCY,
#     farmer_estimates=irrigation_estimate
# )
# plotter = ValidationPlotter()

# for year in range(2018, 2023):
#     ET_blue_per_field = ee.FeatureCollection(
#         f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_m3_ha_yr_per_field_from_dekadal_2018-2023/ET_blue_m3_ha_yr_per_field_from_dekadal_{year}"
#     )
    
#     double_cropping_image = ee.Image(
#         f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{year}"
#     )
    
#     processed_df = validator.process_feature_collection(
#         ET_blue_per_field,
#         double_cropping_image,
#         nutzung_column="nutzung",
#     )
    
#     results = validator.process_validation_data(processed_df)
    
#     output_dir = Path(f"/Users/cooper/Desktop/Hydrosolutions/ETblue-estimation/images/ET_blue_validation")
#     output_dir.mkdir(parents=True, exist_ok=True)
    
#     plotter.create_validation_plots(
#         df=processed_df,
#         validation_results=results,
#         output_dir=output_dir
#     )

In [ ]:
# def filter_dataframe(df: pd.DataFrame, crop_types: List[str]) -> pd.DataFrame:
#     """
#     Filter the DataFrame to include only the specified crop types.

#     Args:
#         df (pd.DataFrame): Input DataFrame with 'ET_blue_m3_ha_yr' and 'nutzung' columns.
#         crop_types (List[str]): List of crop types to include.

#     Returns:
#         pd.DataFrame: Filtered DataFrame.
#     """
#     return df[df["nutzung"].isin(crop_types)]


# def plot_histogram_comparison(
#     calculated_et_blue: pd.DataFrame,
#     farmer_estimates: Dict[str, Tuple[int, int]],
#     output_destination: str,
# ) -> None:
#     """
#     Create a histogram comparison of calculated ET blue vs farmer estimates.

#     Args:
#         calculated_et_blue (pd.DataFrame): DataFrame with 'ET_blue_m3_ha_yr' and 'nutzung' columns.
#         farmer_estimates (Dict[str, Tuple[int, int]]): Dictionary of farmer estimates for each crop type.
#         output_destination (str): Output destination for the plot.
#     """
#     plot_config.set_plot_style()

#     crop_types = list(farmer_estimates.keys())
#     num_crops = len(crop_types)

#     fig, axs = plt.subplots(1, num_crops, figsize=(6 * num_crops, 6))

#     for i, crop in enumerate(crop_types):
#         crop_data = calculated_et_blue[calculated_et_blue["nutzung"] == crop][
#             "ET_blue_m3_ha_yr"
#         ]

#         sns.histplot(crop_data, kde=True, ax=axs[i])
#         axs[i].axvline(
#             farmer_estimates[crop][0],
#             color="r",
#             linestyle="--",
#             label="Farmer's estimate range",
#         )
#         axs[i].axvline(farmer_estimates[crop][1], color="r", linestyle="--")

#         axs[i].set_xlabel("ET Blue (m³/ha/yr)")
#         axs[i].set_ylabel("Frequency")
#         axs[i].set_title(f"{crop}")

#         # Add summary statistics
#         mean_value = crop_data.mean()
#         median_value = crop_data.median()
#         axs[i].axvline(
#             mean_value, color="g", linestyle="-", label=f"Average: {mean_value:.0f}"
#         )

#         axs[i].legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=2)

#     plt.tight_layout()

#     # plt.savefig(output_destination, bbox_inches="tight", dpi=300)
#     plt.subplots_adjust(bottom=0.2)

In [ ]:
# for year in range(2022, 2023):
#     ET_blue_per_field = ee.FeatureCollection(
#         f"projects/thurgau-irrigation/assets/Zuerich/ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022/ET_blue_m3_ha_yr_per_field_from_dekadal_wapor_10m_ZH_2022"
#     )

#     double_cropping_image = ee.Image(
#         f"projects/thurgau-irrigation/assets/Zuerich/crop_vegetation_period_zh_2022"
#     )

#     ET_blue_per_field = add_double_cropping_info(ET_blue_per_field, double_cropping_image)

#     # Filter for single cropping fields
#     ET_blue_per_field = ET_blue_per_field.filter(ee.Filter.eq("isDoubleCropped", 0))

#     ET_blue_per_field_year_df = geemap.ee_to_df(ET_blue_per_field)
#     ET_blue_per_field_year_df = ET_blue_per_field_year_df[
#         ["ET_blue_m3_ha_yr", "NUTZUNG"]
#     ]

#     ET_blue_per_field_year_df["ET_blue_m3_ha_yr"] = (
#         ET_blue_per_field_year_df["ET_blue_m3_ha_yr"] / IRRIGATION_EFFICIENCY
#     )

#     # Filter the DataFrame
#     filtered_df = filter_dataframe(
#         ET_blue_per_field_year_df, list(irrigation_estimate.keys())
#     )

#     filtered_df = filtered_df[filtered_df["ET_blue_m3_ha_yr"] > 0]

#     # Create the plot
#     plot_histogram_comparison(
#         filtered_df,
#         irrigation_estimate,
#         output_destination=f"/Users/cooper/Desktop/Hydrosolutions/ETblue-estimation /images/ET_blue_validation/ET_blue_histogram_comparison_{year}.png",
#     )